In [1]:
PATH_TO_SAMPLE_FILE = "/data/chendian/repo/DCN/data/dcn_train.txt"

In [2]:
lines = [line for line in open(PATH_TO_SAMPLE_FILE, 'r')]
len(lines)

281381

In [8]:
from tqdm import tqdm
from collections import Counter

char_level_pairs = []

for line in tqdm(lines):
    err, cor = line.rstrip().split('\t')[:2]
    err = err.replace(' ', '')
    cor = cor.replace(' ', '')
    if err == cor:
        continue
    else:
        faulty_position = []
        for i, (_e, _c) in enumerate(zip(err, cor)):
            if _e != _c:
                char_level_pairs.append((_e, _c))


ct = Counter(char_level_pairs).most_common()

100%|██████████| 281381/281381 [00:04<00:00, 60302.75it/s]


## char-level confusion

In [12]:
import jieba
import random
from tqdm import tqdm
from copy import deepcopy

# char-level confusion from SpellGCN
cfs_path = '../data/spellGraphs.txt'
char_cfs = {}
for line in open(cfs_path, 'r'):
    l, r, t = line.strip().split('|')
    if t in ['同音同调', '同音异调', '近音异调', '近音同调']:
        t = '近音'
    char_cfs.setdefault(t, {})
    char_cfs[t].setdefault(l, [])
    char_cfs[t][l].append(r)
backup_cfs = deepcopy(char_cfs)
print("char-cfs Loaded", list(char_cfs.keys()))


def char_confusor(char):
    # always take different token
    take = char
    candidates = char_cfs['近音'].get(char, [char])
    if candidates:
        take = random.choice(candidates)
        if take != char:
            char_cfs['近音'][char].remove(take)
    else:
        if backup_cfs['近音'][char]:
            char_cfs['近音'][char] = [_c for _c in backup_cfs['近音'][char]]
    return take


def augment_single_sample(err, cor, confusor):
    faulty_position = []
    for i, (_e, _c) in enumerate(zip(err, cor)):
        if _e != _c:
            faulty_position.append((i, _e, _c))
    for i, e, c in faulty_position:
        assert cor[i] == c
        cor = f"{cor[:i]}{confusor(c)}{cor[i+1:]}"
    return cor

dir_path = '../exp/data/cn/'
# SIGHAN
# src_path = dir_path + 'sighan15/sighan15_train.tsv'
# tgt_path = dir_path + 'sighan15/sighan15_train.augc.tsv'

# Wang271K + SIGHAN
src_path = dir_path + 'Wang271k/dcn_train.tsv'
tgt_path = dir_path + 'Wang271k/dcn_train.augc.tsv'
with open(tgt_path, 'w') as f:
    for line in tqdm(open(src_path, 'r')):
        err, cor = line.strip().split('\t')
        aug_err = augment_single_sample(err, cor, confusor=char_confusor)
        f.write(f"{aug_err}\t{cor}\n")


char-cfs Loaded ['形近', '近音', '同部首同笔画']


272099it [00:03, 72901.51it/s]


## word-level confusion

In [9]:
import jieba
import random
from tqdm import tqdm


def word_confusor(char):
    return random.choice(char_cfs['近音'].get(char, [char]))


def augment_single_sample(err, cor, confusor):
    faulty_position = []
    for i, (_e, _c) in enumerate(zip(err, cor)):
        if _e != _c:
            faulty_position.append((i, _e, _c))
    for i, e, c in faulty_position:
        assert cor[i] == c
        cor = f"{cor[:i]}{confusor(c)}{cor[i+1:]}"
    return cor


src_path = '../exp/data/cn/Wang271k/dcn_train.tsv'
tgt_path = '../exp/data/cn/Wang271k/dcn_train.augc.tsv'
with open(tgt_path, 'w') as f:
    for line in tqdm(open(src_path, 'r')):
        err, cor = line.strip().split('\t')
        aug_err = augment_single_sample(err, cor, confusor=char_confusor)
        f.write(f"{aug_err}\t{cor}\n")


[(('曰', '日'), 13648),
 (('干', '千'), 5936),
 (('末', '未'), 5100),
 (('宫', '官'), 4737),
 (('莫', '其'), 4197),
 (('[', '在'), 3668),
 (('撮', '报'), 3430),
 (('肘', '时'), 2755),
 (('墓', '基'), 2702),
 (('养', '界'), 2495),
 (('玫', '攻'), 2398),
 (('柜', '相'), 2303),
 (('顶', '项'), 2283),
 (('豪', '美'), 2086),
 (('誓', '警'), 1965),
 (('苜', '有'), 1762),
 (('罡', '是'), 1586),
 (('己', '已'), 1558),
 (('耍', '要'), 1504),
 (('他', '她'), 1426),
 (('备', '各'), 1405),
 (('驭', '取'), 1353),
 (('涠', '油'), 1301),
 (('莫', '美'), 1300),
 (('菖', '直'), 1156),
 (('的', '得'), 1152),
 (('旱', '早'), 1137),
 (('天', '夫'), 1131),
 (('找', '我'), 1125),
 (('黄', '美'), 1072)]